<p style="background:#484E6E; color:white; font-size: 2.1em; text-align:center"> 
    <br><b> Lending Club</b><br>
    <br>Data Sampling<br><br>
    
</p>

<p style="text-align: right;">Module 3: Machine Learning<br>
Sprint 3: Unsupervised Learning & Hyperparameter Tuning<br>
Author : Renato Mariano</p>

<h1 style="color:#484E6E">Introduction </h1>

This data science project aims at automating lending decisions for a Lending Club. This dataset was provided direct by turing college on the following link: https://storage.googleapis.com/335-lending-club/lending-club.zip

Similar datasets for the lending club can be found directly on kaggle.

**What We Aim to Achieve**:

- **Exploration**: We will delve into the dataset to extract valuable insights and prepare the data for further analysis.

- **Understanding Data**: We will conduct an Exploratory Data Analysis (EDA) to uncover the underlying characteristics of the dataset and explore relationships between variables.

- **Statistical Insight**: Our journey will include statistical inference to test hypotheses related to the rejected loan applications.

- **Machine Learning Models**: We will develop and refine machine learning models designed to predict whether a loan application is accepted or rejected.

- **Real-world Deployment**: Our final goal is to deploy the best-performing machine learning model on Google Cloud Platform for practical, real-world usage.

- **Recommendations**: Throughout our exploration, we will provide valuable insights and recommendations to enhance our analysis and model performance.

**Specific Objective**:
1. Build a machine learning **model to classify loans as accepted or rejected**.

2. **Predict the loan grade**.

3. **Predict the loan subgrade and interest rate**.

<h1 style="color:#484E6E">Data Sampling</h1>

**This notebook was quickly used only for sampling the data into train, test and validation sets.** Before separation, **duplicated rows were eliminated**.

**Our first sampling** of the data aims at creating a **model to reject of accept loans** (Object 1). 
In that way, the following steps will be followed:
- in the accepted dataframe only the features found in rejected will be maintened.
- EDA will be performed on the merged training data.
- Pipelines and modeling section will be defined.

**After, we will perform EDA on the remaining features of the accepted dataframe and accomplish Objectives 2 and 3**.

Due to the large size of the dataset, we used the **polars library in for the sampling**.

In [1]:
import polars as pl
from time import time
from sklearn.model_selection import train_test_split
import gc

<h2 style="color:#484E6E">Rejected Dataframe</h2>

In [2]:
start = time()
rejected_df = pl.read_csv("data/rejected_2007_to_2018q4.csv", infer_schema_length=10000, ignore_errors=True)
end = time()
print("Read csv with polars: ",(end-start),"sec\n")
print(f"Shape of the dataframe: {rejected_df.shape}")
rejected_df.head(5)

Read csv with polars:  34.066680908203125 sec

Shape of the dataframe: (27648741, 9)


Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
f64,str,str,f64,str,str,str,str,f64
1000.0,"""2007-05-26""","""Wedding Covere…",693.0,"""10%""","""481xx""","""NM""","""4 years""",0.0
1000.0,"""2007-05-26""","""Consolidating …",703.0,"""10%""","""010xx""","""MA""","""< 1 year""",0.0
11000.0,"""2007-05-27""","""Want to consol…",715.0,"""10%""","""212xx""","""MD""","""1 year""",0.0
6000.0,"""2007-05-27""","""waksman""",698.0,"""38.64%""","""017xx""","""MA""","""< 1 year""",0.0
1500.0,"""2007-05-27""","""mdrigo""",509.0,"""9.43%""","""209xx""","""MD""","""< 1 year""",0.0


In [3]:
duplicated = rejected_df.filter(rejected_df.is_duplicated())
print(f"The number of duplicated values is: {duplicated.shape[0]}")
duplicated.head(10)

The number of duplicated values is: 299488


Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
f64,str,str,f64,str,str,str,str,f64
30000.0,"""2007-11-21""","""debt_consolida…",null,"""100%""","""902xx""","""FL""","""10+ years""",0.0
30000.0,"""2007-11-21""","""debt_consolida…",null,"""100%""","""902xx""","""FL""","""10+ years""",0.0
5000.0,"""2007-11-28""","""other""",646.0,"""13.36%""","""085xx""","""NJ""","""10+ years""",0.0
5000.0,"""2007-11-28""","""other""",646.0,"""13.36%""","""085xx""","""NJ""","""10+ years""",0.0
9000.0,"""2008-03-26""","""major_purchase…",null,"""0%""","""916xx""","""CA""","""< 1 year""",0.0
9000.0,"""2008-03-26""","""major_purchase…",null,"""0%""","""916xx""","""CA""","""< 1 year""",0.0
9000.0,"""2008-03-26""","""major_purchase…",null,"""0%""","""916xx""","""CA""","""< 1 year""",0.0
7000.0,"""2008-06-19""",""" """,667.0,"""0%""","""750xx""","""TX""","""2 years""",0.0
7000.0,"""2008-06-19""",""" """,667.0,"""0%""","""750xx""","""TX""","""2 years""",0.0


In [4]:
rejected_df_no_dupli = rejected_df.unique(keep='none')
rejected_df_no_dupli.filter(rejected_df_no_dupli.is_duplicated())

Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
f64,str,str,f64,str,str,str,str,f64


In [5]:
del rejected_df
gc.collect()

20

Although there is no client id on this dataframe, we can see that **these requests are very likely to come from the same client, thus they were dropped**.

Bellow, some slight **changes to the columns' names** are applied and also a **removal of the percentage signs** in the Debt-To-Income Ratio.

In [6]:
dict_cols = {
    "Amount Requested": "Amount_Requested",
    "Loan Title": "Loan_Title",
    "Debt-To-Income Ratio": "Debt-To-Income_Ratio",
    "Zip Code": "Zip_Code",
    "Employment Length": "Employment_Length",
    "Policy Code": "Policy_Code",
}
rejected_df_no_dupli = rejected_df_no_dupli.rename(dict_cols)

In [7]:
rejected_df_no_dupli = rejected_df_no_dupli.with_columns(pl.col("Debt-To-Income_Ratio").str.replace(r"%", "").cast(pl.Float64))
rejected_df_no_dupli.head()

Amount_Requested,Application Date,Loan_Title,Risk_Score,Debt-To-Income_Ratio,Zip_Code,State,Employment_Length,Policy_Code
f64,str,str,f64,f64,str,str,str,f64
1000.0,"""2007-05-26""","""Wedding Covere…",693.0,10.0,"""481xx""","""NM""","""4 years""",0.0
1000.0,"""2007-05-26""","""Consolidating …",703.0,10.0,"""010xx""","""MA""","""< 1 year""",0.0
11000.0,"""2007-05-27""","""Want to consol…",715.0,10.0,"""212xx""","""MD""","""1 year""",0.0
6000.0,"""2007-05-27""","""waksman""",698.0,38.64,"""017xx""","""MA""","""< 1 year""",0.0
1500.0,"""2007-05-27""","""mdrigo""",509.0,9.43,"""209xx""","""MD""","""< 1 year""",0.0


<h2 style="color:#484E6E">Accepted Dataframe</h2>

In [8]:
import polars as pl
from time import time
from sklearn.model_selection import train_test_split
import gc

In [9]:
start = time()
accepted_df = pl.read_csv("data/accepted_2007_to_2018q4.csv", infer_schema_length=10000, ignore_errors=True)
end = time()
print("Read csv with polars: ",(end-start),"sec\n")
print(f"Shape of the dataframe: {accepted_df.shape}")
accepted_df.head(5)

Read csv with polars:  31.44804286956787 sec

Shape of the dataframe: (2260701, 151)


id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,…,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
i64,str,f64,f64,f64,str,f64,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,f64,f64,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64
68407277,null,3600.0,3600.0,3600.0,""" 36 months""",13.99,123.03,"""C""","""C4""","""leadman""","""10+ years""","""MORTGAGE""",55000.0,"""Not Verified""","""Dec-2015""","""Fully Paid""","""n""","""https://lendin…",null,"""debt_consolida…","""Debt consolida…","""190xx""","""PA""",5.91,0.0,"""Aug-2003""",675.0,679.0,1.0,30.0,null,7.0,0.0,2765.0,29.7,13.0,…,13734.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Cash""","""N""",null,null,null,null,null,null
68355089,null,24700.0,24700.0,24700.0,""" 36 months""",11.99,820.28,"""C""","""C1""","""Engineer""","""10+ years""","""MORTGAGE""",65000.0,"""Not Verified""","""Dec-2015""","""Fully Paid""","""n""","""https://lendin…",null,"""small_business…","""Business""","""577xx""","""SD""",16.06,1.0,"""Dec-1999""",715.0,719.0,4.0,6.0,null,22.0,0.0,21470.0,19.2,38.0,…,24667.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Cash""","""N""",null,null,null,null,null,null
68341763,null,20000.0,20000.0,20000.0,""" 60 months""",10.78,432.66,"""B""","""B4""","""truck driver""","""10+ years""","""MORTGAGE""",63000.0,"""Not Verified""","""Dec-2015""","""Fully Paid""","""n""","""https://lendin…",null,"""home_improveme…",null,"""605xx""","""IL""",10.78,0.0,"""Aug-2000""",695.0,699.0,0.0,null,null,6.0,0.0,7869.0,56.2,18.0,…,14877.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Cash""","""N""",null,null,null,null,null,null
66310712,null,35000.0,35000.0,35000.0,""" 60 months""",14.85,829.9,"""C""","""C5""","""Information Sy…","""10+ years""","""MORTGAGE""",110000.0,"""Source Verifie…","""Dec-2015""","""Current""","""n""","""https://lendin…",null,"""debt_consolida…","""Debt consolida…","""076xx""","""NJ""",17.06,0.0,"""Sep-2008""",785.0,789.0,0.0,null,null,13.0,0.0,7802.0,11.6,17.0,…,18000.0,null,null,null,null,null,null,null,null,null,null,null,null,null,"""N""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Cash""","""N""",null,null,null,null,null,null
68476807,null,10400.0,10400.0,10400.0,""" 60 months""",22.45,289.91,"""F""","""F1""","""Contract Speci…","""3 years""","""MORTGAGE""",104433.0,"""Source Verifie…","""Dec-2015""","""Fully Paid""","""n""","""https://lendin…",null,"""major_purchase…","""Major purch

In [10]:
duplicated = accepted_df.filter(accepted_df.is_duplicated())
print(f"The number of duplicated values is: {duplicated.shape[0]}")
duplicated.head(4)

The number of duplicated values is: 33


id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,…,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
i64,str,f64,f64,f64,str,f64,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,f64,f64,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In the case of the accepted dataframe, **all of the duplicated values represented actually blank lines** on the dataframe that **were also dropped**.

In [11]:
accepted_df_no_dupli = accepted_df.unique(keep='none')
accepted_df_no_dupli.filter(accepted_df_no_dupli.is_duplicated())

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,…,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
i64,str,f64,f64,f64,str,f64,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,f64,f64,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64


<h3 style="color:#484E6E">Export Accepted Dataframe</h3>

For the use on the 2nd and 3rd Objectives of this project (loan grade, loan subgrade and interest rate), the accepted dataframe will also be separated and exported.

In [13]:
train_size = 0.4
val_test_size = 0.3

#accepted_train, accepted_temp = train_test_split(accepted_df_no_dupli, test_size=1 - train_size, random_state=1)
#accepted_validation, accepted_test = train_test_split(accepted_temp, test_size=0.5, random_state=1)

In [14]:
#print(accepted_train.shape, accepted_test.shape, accepted_validation.shape)

In [15]:
#accepted_train.write_csv("data/exported/accepted_train.csv")
#accepted_test.write_csv("data/exported/accepted_test.csv")
#accepted_validation.write_csv("data/exported/accepted_validation.csv")

Remove defined dataframes from memory

In [16]:
#del accepted_train, accepted_test, accepted_validation
#gc.collect()

As mentioned above, **we will keep only the features that have correspondent in the rejected dataframe**.

In [12]:
accepted_df_clean = accepted_df_no_dupli[["loan_amnt", "title", "fico_range_low", "fico_range_high", "dti", "zip_code", "addr_state", "emp_length", "policy_code"]]
accepted_df_clean.head()

loan_amnt,title,fico_range_low,fico_range_high,dti,zip_code,addr_state,emp_length,policy_code
f64,str,f64,f64,f64,str,str,str,f64
3600.0,"""Debt consolida…",675.0,679.0,5.91,"""190xx""","""PA""","""10+ years""",1.0
24700.0,"""Business""",715.0,719.0,16.06,"""577xx""","""SD""","""10+ years""",1.0
20000.0,null,695.0,699.0,10.78,"""605xx""","""IL""","""10+ years""",1.0
35000.0,"""Debt consolida…",785.0,789.0,17.06,"""076xx""","""NJ""","""10+ years""",1.0
10400.0,"""Major purchase…",695.0,699.0,25.37,"""174xx""","""PA""","""3 years""",1.0


The Application Date column in rejected has no correspondent in the accepted dataframe, it will be dropped!

In [13]:
dict_cols = {
    "loan_amnt": "Amount_Requested",
    "title": "Loan_Title",
    "dti": "Debt-To-Income_Ratio",
    "zip_code": "Zip_Code",
    "addr_state": "State",
    "emp_length": "Employment_Length",
    "policy_code": "Policy_Code",
}
accepted_df_clean = accepted_df_clean.rename(dict_cols)
accepted_df_clean.head()

Amount_Requested,Loan_Title,fico_range_low,fico_range_high,Debt-To-Income_Ratio,Zip_Code,State,Employment_Length,Policy_Code
f64,str,f64,f64,f64,str,str,str,f64
3600.0,"""Debt consolida…",675.0,679.0,5.91,"""190xx""","""PA""","""10+ years""",1.0
24700.0,"""Business""",715.0,719.0,16.06,"""577xx""","""SD""","""10+ years""",1.0
20000.0,null,695.0,699.0,10.78,"""605xx""","""IL""","""10+ years""",1.0
35000.0,"""Debt consolida…",785.0,789.0,17.06,"""076xx""","""NJ""","""10+ years""",1.0
10400.0,"""Major purchase…",695.0,699.0,25.37,"""174xx""","""PA""","""3 years""",1.0


In [14]:
del accepted_df, accepted_df_no_dupli
gc.collect()

0

The FICO scores presented in accepted are high and low. We will translate that into Risk_Score with an average between the two values.

Besides, we also create the Application Result as "Accepted" for this dataframe.

In [15]:
accepted_df_clean = accepted_df_clean.with_columns(Risk_Score = accepted_df_clean[["fico_range_high", "fico_range_low"]].mean(axis=1))

accepted_df_clean = accepted_df_clean.select(
                                            ['Amount_Requested',
                                            'Loan_Title',
                                            'Risk_Score',
                                            'Debt-To-Income_Ratio',
                                            'Zip_Code',
                                            'State',
                                            'Employment_Length',
                                            'Policy_Code']
                                            )

accepted_df_clean.head()

Amount_Requested,Loan_Title,Risk_Score,Debt-To-Income_Ratio,Zip_Code,State,Employment_Length,Policy_Code
f64,str,f64,f64,str,str,str,f64
3600.0,"""Debt consolida…",677.0,5.91,"""190xx""","""PA""","""10+ years""",1.0
24700.0,"""Business""",717.0,16.06,"""577xx""","""SD""","""10+ years""",1.0
20000.0,null,697.0,10.78,"""605xx""","""IL""","""10+ years""",1.0
35000.0,"""Debt consolida…",787.0,17.06,"""076xx""","""NJ""","""10+ years""",1.0
10400.0,"""Major purchase…",697.0,25.37,"""174xx""","""PA""","""3 years""",1.0


In [16]:
accepted_df_clean = accepted_df_clean.with_columns(Application_Result = pl.lit("Accepted"))
accepted_df_clean.head()

Amount_Requested,Loan_Title,Risk_Score,Debt-To-Income_Ratio,Zip_Code,State,Employment_Length,Policy_Code,Application_Result
f64,str,f64,f64,str,str,str,f64,str
3600.0,"""Debt consolida…",677.0,5.91,"""190xx""","""PA""","""10+ years""",1.0,"""Accepted"""
24700.0,"""Business""",717.0,16.06,"""577xx""","""SD""","""10+ years""",1.0,"""Accepted"""
20000.0,null,697.0,10.78,"""605xx""","""IL""","""10+ years""",1.0,"""Accepted"""
35000.0,"""Debt consolida…",787.0,17.06,"""076xx""","""NJ""","""10+ years""",1.0,"""Accepted"""
10400.0,"""Major purchase…",697.0,25.37,"""174xx""","""PA""","""3 years""",1.0,"""Accepted"""


<h2 style="color:#484E6E">Merging Dataframes</h2>

In [17]:
rejected_df_no_dupli = rejected_df_no_dupli.with_columns(Application_Result = pl.lit("Rejected"))
rejected_df_no_dupli = rejected_df_no_dupli.drop("Application Date")
rejected_df_no_dupli.head()

Amount_Requested,Loan_Title,Risk_Score,Debt-To-Income_Ratio,Zip_Code,State,Employment_Length,Policy_Code,Application_Result
f64,str,f64,f64,str,str,str,f64,str
1000.0,"""Wedding Covere…",693.0,10.0,"""481xx""","""NM""","""4 years""",0.0,"""Rejected"""
1000.0,"""Consolidating …",703.0,10.0,"""010xx""","""MA""","""< 1 year""",0.0,"""Rejected"""
11000.0,"""Want to consol…",715.0,10.0,"""212xx""","""MD""","""1 year""",0.0,"""Rejected"""
6000.0,"""waksman""",698.0,38.64,"""017xx""","""MA""","""< 1 year""",0.0,"""Rejected"""
1500.0,"""mdrigo""",509.0,9.43,"""209xx""","""MD""","""< 1 year""",0.0,"""Rejected"""


In [18]:
full_df = pl.concat([accepted_df_clean, rejected_df_no_dupli])
print(full_df.head())
print(full_df.tail())

shape: (5, 9)
┌────────────┬────────────┬────────────┬───────────┬───┬───────┬───────────┬───────────┬───────────┐
│ Amount_Req ┆ Loan_Title ┆ Risk_Score ┆ Debt-To-I ┆ … ┆ State ┆ Employmen ┆ Policy_Co ┆ Applicati │
│ uested     ┆ ---        ┆ ---        ┆ ncome_Rat ┆   ┆ ---   ┆ t_Length  ┆ de        ┆ on_Result │
│ ---        ┆ str        ┆ f64        ┆ io        ┆   ┆ str   ┆ ---       ┆ ---       ┆ ---       │
│ f64        ┆            ┆            ┆ ---       ┆   ┆       ┆ str       ┆ f64       ┆ str       │
│            ┆            ┆            ┆ f64       ┆   ┆       ┆           ┆           ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════╪═══════════╪═══════════╪═══════════╡
│ 3600.0     ┆ Debt conso ┆ 677.0      ┆ 5.91      ┆ … ┆ PA    ┆ 10+ years ┆ 1.0       ┆ Accepted  │
│            ┆ lidation   ┆            ┆           ┆   ┆       ┆           ┆           ┆           │
│ 24700.0    ┆ Business   ┆ 717.0      ┆ 16.06     ┆ … ┆ SD    ┆ 10+ years ┆ 

In [19]:
del accepted_df_clean, rejected_df_no_dupli
gc.collect()

0

In [26]:
full_df["Application_Result"].value_counts().select(
    pl.col('Application_Result'), 
    pl.col('counts'),
    (pl.col('counts')/pl.col('counts').sum()*100).round(2).alias('percent')
)

Application_Result,counts,percent
str,u32,f64
"""Accepted""",2260668,7.63
"""Rejected""",27349253,92.37


This Dataset is very imbalnced (92% of rejected applications). There was a try to split the data with a stratified method, but due to memory alocation it did not work. As we will go through the next notebook, we will see that similar proportions were obtained in this process anyways.

<h2 style="color:#484E6E">Export Merged Dataframe</h2>

Let's separate our train, test and validation sets.

In [1]:
train_size = 0.4
val_test_size = 0.3

#full_train, full_temp = train_test_split(full_df, test_size=1 - train_size, random_state=1)

In [26]:
#full_train.write_csv("data/exported/full_train.csv")

#del full_train
#gc.collect()

0

In [27]:
#full_validation, full_test = train_test_split(full_temp, test_size=0.5, random_state=1)

In [28]:
#full_test.write_csv("data/exported/full_test.csv")
#full_validation.write_csv("data/exported/full_validation.csv")

We will continue this work in the 2nd notebook for this project. 02_EDA_model01_LendingClub